In [1]:
import sys
sys.path.append('/Users/Ziqi/Desktop/developer/mgwr/')
from mgwr.gwr import GWR
from mgwr.FastGWR import FastGWR
from mgwr.sel_bw import Sel_BW
import pandas as pd
import numpy as np

In [2]:
zillow = pd.read_csv("/Users/Ziqi/Desktop/developer/gwr-hpc/zillow_15k.csv",sep=',')
zillow.head()
y = zillow.value.as_matrix().reshape(-1,1)
X = zillow.iloc[:,3:].as_matrix()
k = zillow.shape[1]
u = zillow.utmX
v = zillow.utmY
n = zillow.shape[0]
coords = list(zip(u,v))

In [3]:
#Adaptive Bisquare

In [4]:
%%time 
fast_rslt = FastGWR(coords,y,X).fit(nproc=2) #FastGWR fitting with optimal bw seaching

CPU times: user 427 ms, sys: 29.1 ms, total: 456 ms
Wall time: 3min 45s


In [5]:
%%time 
bw = Sel_BW(coords,y,X).search() #MGWR fitting with optimal bw seaching
rslt = GWR(coords,y,X,bw=bw).fit()

CPU times: user 5min 54s, sys: 3min 56s, total: 9min 51s
Wall time: 14min 46s


In [6]:
#tr_S
print(rslt.tr_S)
print(fast_rslt.tr_S)
#AICc
print(rslt.aicc)
print(fast_rslt.aicc)
#Sigma2
print(rslt.sigma2)
print(fast_rslt.sigma2)
#Estimtaes
print(np.allclose(fast_rslt.params, rslt.params))
print(np.allclose(fast_rslt.bse, rslt.bse))
print(np.allclose(fast_rslt.predy, rslt.predy.reshape(-1)))

1053.314183297261
1053.314183297261
186063.51147149547
186063.5114714955
13199.728882664554
13199.72888266456
True
True
True


In [7]:
#Fixed Gaussian

In [17]:
%%time
fast_rslt = FastGWR(coords,y,X,fixed=True,bw=10000).fit(nproc=2) #FastGWR fitting with optimal bw seaching

CPU times: user 433 ms, sys: 79.4 ms, total: 513 ms
Wall time: 18.8 s


In [18]:
%%time
rslt = GWR(coords,y,X,bw=10000,fixed=True,kernel='gaussian').fit()

CPU times: user 1min 6s, sys: 17.6 s, total: 1min 24s
Wall time: 2min


In [19]:
#tr_S
print(rslt.tr_S)
print(fast_rslt.tr_S)
#AICc
print(rslt.aicc)
print(fast_rslt.aicc)
#Sigma2
print(rslt.sigma2)
print(fast_rslt.sigma2)
#Estimtaes
print(np.allclose(fast_rslt.params, rslt.params))
print(np.allclose(fast_rslt.bse, rslt.bse))
print(np.allclose(fast_rslt.predy, rslt.predy.reshape(-1)))

38.90050865124418
38.90050865124418
188446.97760932142
188446.9776093214
16689.196891303494
16689.196891303494
True
True
True
